In [1]:
from data import gdf
import leafmap.foliumap as leafmap
import geopandas as gpd

In [3]:
# service_boundary = gdf.pipe(lambda df:df.loc[df['label'] == "Frick Unit North Service Area"])
# service_boundary.plot()
# clip = gdf.clip(service_boundary)
# print(gdf.shape)
# print(clip.shape)
# clip.plot()

In [4]:
# clip = gdf.clip_by_rect(*bounds)
# # clip = gdf.clip(service_boundary)
# clip = gdf.clip(service_boundary)
# # filtered_gdf = gdf.iloc[clip.loc[~clip.geometry.is_empty].index]
# full_gdf = clip
# # print(filtered_gdf.shape)
# # filtered_gdf.plot()
# print(full_gdf.shape)
# full_gdf.plot()


In [13]:
import folium

def make_map(full_gdf):
	lines = ['Proposed Pipeline', 'AEWSD North Canal', 'FFPPP Discharge Pipeline',"AEWSD Alignments"]
	filled_polygons = ['Frick Unit Service Area']
	hollow_polygons = ['District Boundary']
	points = ['Proposed Turnout']

	clip_layers = ['Proposed Turnout', 'Proposed Pipeline',"Frick Unit Service Area","AEWSD Alignments"]
	service_boundary = full_gdf.pipe(lambda df:df.loc[df['label'] == "Frick Unit North Service Area"])
	clip = lambda gdf: gdf.clip(service_boundary)
		
	m = leafmap.Map(
		# tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
		# attr="Esri",
		google_map="HYBRID",
		min_zoom=3,
		# max_zoom=12,

		# min_lat=bounds[2],
		# max_lat=bounds[3],
		# min_lon=bounds[0],
		# max_lon=bounds[1],
		# max_bounds=True,
		
		# zoom_control=False,
		draw_control=False,
		search_control=False,

	)

	for layer in hollow_polygons:
		gdf = full_gdf[full_gdf['layer']==layer]
		if layer in clip_layers:
			gdf = clip(gdf)
		color = gdf['color'].unique()[0]
		m.add_gdf(
			gdf,
			layer_name=layer,
			style={
				'color':color,
				'fillColor':"none",
				},
			)

	for layer in filled_polygons:
		gdf = full_gdf[full_gdf['layer']==layer]
		if layer in clip_layers:
			gdf = clip(gdf)
		color = gdf['color'].unique()[0]
		m.add_gdf(
			gdf,
			layer_name=layer,
			style={
				# 'color':color,
				'color':"none",
				'fillColor':color,
				"tooltip":"label",
				},
				# tooltip="label",
			zoom_to_layer=True,
			)

	for layer in lines:
		gdf = full_gdf[full_gdf['layer']==layer]
		if layer in clip_layers:
			gdf = clip(gdf)
		color = gdf['color'].unique()[0]
		m.add_gdf(
			gdf,
			layer_name=layer,
			# tooltip="label",
			style={'color': color},
			)

	for layer in points:
		gdf = full_gdf[full_gdf['layer']==layer]
		if layer in clip_layers:
			gdf = clip(gdf)
		color = gdf['color'].unique()[0]
		for i,y in gdf.iterrows():
			folium.Circle(
				radius=30,
				location=[y.geometry.y,y.geometry.x],
				color=y['color'],
				fill=True,
				# tooltip=[y["label"]],
				tooltip="Proposed Turnout",
				).add_to(m)
	m.add_legend(
		title="Legend",
		legend_dict={
			"Proposed Pipeline": "orange",
			# "AEWSD Alignments": "orange",
			"AEWSD North Canal": "#0000ff",
			"FFPPP Discharge Pipeline": "red",
			"Frick Unit Service Area": "salmon",
			"District Boundary": "#000000",
			"Proposed Turnout": "#ffff00",
		}
	)


	# m.fit_bounds(bounds=ll_bounds,max_zoom=12)
	# m.set_max_bounds(ll_bounds)
	return m
# filtered_gdf = gdf.clip(bounds)
# filtered_gdf = gdf.clip(bounds)
m = make_map(gdf)
m

In [26]:
# def filter_bbox(gdf,bounds):
# 	# clip = gdf.clip_by_rect(*bounds)
# 	clip = gdf.clip_by_rect(
# 		xmin=bounds[0],
# 		xmax=bounds[1],
# 		ymin=bounds[2],
# 		ymax=bounds[3],
# 	)
# 	# print(clip.shape)
# 	print(clip.loc[~clip.geometry.is_empty])
	
	
# 	filtered_gdf = gdf.iloc[clip.loc[~clip.geometry.is_empty].index]
# 	return filtered_gdf

# bounds = [
# 	# left
# 	-118.93,
# 	# right
# 	-118.80,
# 	# bottom
# 	35.25,
# 	# top
# 	35.35,
# 	]

# filtered_gdf = filter_bbox(gdf,bounds)
# print(filtered_gdf.shape)
# # filtered_gdf.head()
# filtered_gdf.plot()


In [16]:
gdf.clip_by_rect(*bounds)

0      LINESTRING (-118.88750 35.32172, -118.90981 35...
1      LINESTRING (-118.88752 35.31080, -118.90544 35...
2      LINESTRING (-118.88748 35.30355, -118.88297 35...
3      LINESTRING (-118.88748 35.29623, -118.90537 35...
4      LINESTRING (-118.86956 35.28903, -118.86963 35...
                             ...                        
553                          POINT (-118.87886 35.25655)
554                          POINT (-118.88294 35.30350)
555                          POINT (-118.88743 35.29265)
556                          POINT (-118.87870 35.27143)
557                          POINT (-118.88749 35.31072)
Length: 582, dtype: geometry

In [12]:
m.to_html("map.html")